In [1]:
import geopandas as gpd
from shapely.geometry import mapping
import numpy as np

In [41]:
geo_data = gpd.read_file('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')

In [42]:
geo_data.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,0,2,Indonesia,IDN,0,2,Sovereign country,1,Indonesia,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Admin-0 country,0,3,Malaysia,MYS,0,2,Sovereign country,1,Malaysia,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Admin-0 country,0,2,Chile,CHL,0,2,Sovereign country,1,Chile,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Admin-0 country,0,3,Bolivia,BOL,0,2,Sovereign country,1,Bolivia,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Admin-0 country,0,2,Peru,PER,0,2,Sovereign country,1,Peru,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [48]:
columns = [
    "ADMIN",
    "ADM0_A3",
    "CONTINENT",
    "geometry"
]

geo_data[columns].head()

,ADMIN,ADM0_A3,CONTINENT,geometry
0,Indonesia,IDN,Asia,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Malaysia,MYS,Asia,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Chile,CHL,South America,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Bolivia,BOL,South America,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Peru,PER,South America,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [71]:
include_list = [
    "Czechia",
    "Austria",
    "Germany",
    "Poland",
    "Slovakia",
    "Hungary",
    "Switzerland",
    "Italy",
    "France",
    "Liechtenstein",
    "Belgium",
    "Netherlands",
    "Luxembourg",
    "Netherlands"
]
remove_list = [
]

country_coords = {}
# upravit tak, aby zpracoval spravne MULTIPOLYGON - zatim bere pouze nejvetsi polygon

# (geo_data.ADMIN.isin(include_list)) & (~geo_data.ADMIN.isin(remove_list))
for idx, row in geo_data[(geo_data.ADMIN.isin(include_list)) & (~geo_data.ADMIN.isin(remove_list))].iterrows():
    geojson_obj = mapping(row['geometry'])
    geo_coords = geojson_obj["coordinates"]
    idx = np.argmax([len(coords[0]) for coords in geo_coords])
    country_coords[row['ADMIN']] = geo_coords[idx]

In [72]:
class Country:
    def __init__(self, country, country_code, continent, coordinates):
        self.country = country
        self.country_code = country_code
        self.continent = continent
        self.coordinates = coordinates

In [73]:
country_coord_cleansed = {}
for country_name, country_coord in country_coords.items():
    while len(country_coord) < 5:
        country_coord = country_coord[0]
    country_coord_cleansed[country_name] = country_coord

countries = {}
#(geo_data.ADMIN.isin(include_list)) & (~geo_data.ADMIN.isin(remove_list))
for idx, row in geo_data[(geo_data.ADMIN.isin(include_list)) & (~geo_data.ADMIN.isin(remove_list))].iterrows():
    countries[row['ADMIN']] = {
        'country': row['ADMIN'],
        'code': row['ADM0_A3'],
        'continent': row['CONTINENT'],
        'coordinates': country_coord_cleansed[row['ADMIN']]
    }
    
countries

{'France': {'country': 'France',
  'code': 'FRA',
  'continent': 'Europe',
  'coordinates': ((2.521799927545686, 51.08754088371883),
   (2.5370333250000954, 51.06461029100008),
   (2.547058553000113, 51.0204786180001),
   (2.5569804280001165, 51.00117747000006),
   (2.592740519000131, 50.97611440000013),
   (2.6084501540000815, 50.96112823500006),
   (2.6116540930000554, 50.941232809000056),
   (2.6057629800000655, 50.93523834300002),
   (2.582715291000113, 50.92066558900008),
   (2.5769275310001376, 50.91151886000007),
   (2.5966679280001017, 50.86764556900006),
   (2.595531046000076, 50.861134339),
   (2.5877795820000813, 50.85002390600006),
   (2.5867460530000983, 50.845424704000024),
   (2.589949992000072, 50.84201405900011),
   (2.6020422770000664, 50.83808664900003),
   (2.6065898030000767, 50.83483103400006),
   (2.6151680910000152, 50.822170309000015),
   (2.620439087000136, 50.81648590200005),
   (2.6277771400001484, 50.814418844),
   (2.6424532470000486, 50.81245513900008),
 

In [74]:
import json

with open('data/countries.json', 'w') as outfile:
    json.dump(countries, outfile)